In [1]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("indonlp/cendol-mt5-small-chat")
model = AutoModelForSeq2SeqLM.from_pretrained("indonlp/cendol-mt5-small-chat")

c:\Users\Frederick\anaconda3\envs\EmbeddingLearning\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import PyPDF2

def extract_text_from_pdf(pdf_file):
    with open(pdf_file, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page in reader.pages:
            text += page.extract_text()
    return text

In [8]:
from sentence_transformers import SentenceTransformer

EmbeddingModel = SentenceTransformer('paraphrase-MiniLM-L6-v2')

def generate_embeddings(text):
    sentences = text.split(". ")  
    embeddings = EmbeddingModel.encode(sentences, convert_to_tensor=True)
    return sentences, embeddings


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\Frederick\anaconda3\lib\site-packages\transformers\utils\import_utils.py", line 1364, in _get_module
    {0} requires the timm library but it was not found in your environment. You can install it with pip:
  File "c:\Users\Frederick\anaconda3\lib\importlib\__init__.py", line 126, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "<frozen importlib._bootstrap>", line 1050, in _gcd_import
  File "<frozen importlib._bootstrap>", line 1027, in _find_and_load
  File "<frozen importlib._bootstrap>", line 1006, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 688, in _load_unlocked
  File "<frozen importlib._bootstrap_external>", line 883, in exec_module
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "c:\Users\Frederick\anaconda3\lib\site-packages\transformers\trainer_callback.py", line 28, in <module>
    from .training_args import TrainingAr